# Imports

In [32]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
!export PYSPARK_PYTHON=python
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Fetched 338 kB in 5s (64.4 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
18 packages can be upgraded. Run 'apt list --upgradable' to see them.
openjdk-8-jdk-headless is already the 

In [33]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import math
from datetime import datetime

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [34]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [35]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Inicialización de los RDD

In [36]:
id1="17SvnFyBNyY018rxrRnOLGp9YaZfeRknp"
downloaded1 = drive.CreateFile({'id': id1})
downloaded1.GetContentFile('googleplaystore.csv')

id2="1DBCzFlJWzdVirjQmoyTaH3GekLTzrznm"
downloaded2 = drive.CreateFile({'id': id2})
downloaded2.GetContentFile('googleplaystore_user_reviews.csv')

In [37]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

sqlContext = SQLContext(sc)

apps_df = sqlContext.read.csv('googleplaystore.csv', header=True, inferSchema=True, mode='DROPMALFORMED')
apps_rdd = apps_df.rdd

rev_df = sqlContext.read.csv('googleplaystore_user_reviews.csv', header=True, inferSchema=True, mode='DROPMALFORMED')
rev_rdd = rev_df.rdd

In [38]:
# Uso esto en varias preguntas, así que lo defino en "global"
def to_float(str):
  try:
    return float(str)
  except:
    return math.nan

red = '\033[91m'
yellow = '\033[93m'
white = '\033[97m'

# Pregunta #8
[*ir al notebook dedicado*](https://colab.research.google.com/drive/1XS2VOeU1bw3Mu7kNDxQunsifflG_zand?usp=drive_link)

> *Devolver la aplicación más polarizante. tener en cuenta el promedio de polarización de las reviews de cada aplicación.*

**Respuesta:** Homework (promedio de 1.0)

**Notas:**
- Medio rara la respuesta, debe ser de esas que tienen 1 review
- Soy de la opinión de que debería tomar el promedio del *valor absoluto* de las polaridades, pero [dijeron en slack](https://organizacinde-ea47838.slack.com/archives/C02DN3BCUHJ/p1695920493443789) que como era un ejercicio de 1 estrella no hacía falta complicarse la vida (igual me fijé y el resultado no cambia)
- Lo corrí con `%%timeit` *bastantes veces* y la performance usando `reduce()` y usando `max()` es *prácticamente igual*, pero me dijeron que use reduce y aprendí en algo3 que es mejor la nota que ser feliz (?
- La función `to_float()` es un poco bruta para mi gusto pero es sencilla y hace exactamente lo que quiero (pasarme las cosas a float y volar todo lo que no sea un float)

In [39]:
rev_rdd \
  .map(lambda x: (x[0], (to_float(x[3]), 1))) \
  .filter(lambda x: not math.isnan(x[1][0])) \
  .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])) \
  .map(lambda x: (x[0], x[1][0]/x[1][1])) \
  .reduce(lambda x,y: x if x[1]>y[1] else y)

('HomeWork', 1.0)

# Pregunta #12
[*ir al notebook dedicado*](https://colab.research.google.com/drive/1DPnaJJu8kAVZIIdKmFykiCx0X72Kv6vy?usp=drive_link)

> *Juan tiene un dispositivo un poco viejo con Android 3.9 y ya no lo puede actualizar. Quiere descargar apps del género de “Arte y Diseño” pero no sabe cuáles son válidas para su versión de Android. Ayúdalo diciéndole cuántas apps puede descargarse y mencionale 3 de ellas con su respectivo Rating*

**Respuesta:** se puede descargar 11 apps, ver la salida para las 3 apps mencionadas.

**Notas:**
- Solo me interesaban en verdad los 2 primeros números de la versión de android , por lo que convertirlo a float me bastaba
- Btw, en pandas me fijé cuáles eran las posibles versiones (justo tenía uno de los ej con eso) y ví que el casteo que estoy haciendo no me rompe nada (Los únicos NaN de hecho son los "Varies with device", y todas las versiones que son rangos ni pasan por la 3.9)
- Corrí un par de veces el bloque con `%%timeit` y con el cache funciona *un poquito* más rápido
- El primer map y reduceByKey me quitan las apps duplicadas. Hago básicamente lo mismo (map a dupla nombre - campos que quiero y reduceByKey de agarrar uno de los 2) en el resto de los ejercicios que usan el dataset de apps. No tengo ningún criterio para seleccionar a uno de los duplicados, así que agarro uno cualquiera

In [40]:
def ver_to_float(str):
  try:
    return float(str[:3])
  except:
    return math.nan

ans_rdd = apps_rdd \
  .map(lambda x: (x[0], (x[9], x[12], x[2]))) \
  .reduceByKey(lambda x,y: x) \
  .map(lambda x: (x[0], x[1][0], ver_to_float(x[1][1]), x[1][2])) \
  .filter(lambda x: x[2] <= 3.9 and "Art & Design" in x[1]) \
  .cache()

total = ans_rdd.count()
apps = ans_rdd.take(3)

# Para tener salida bonita, no relacionado al ej
print(f"{total} apps encontradas, entre ellas:")

for app in apps:
  print(red+"================================")
  print(f"{yellow}App: {white}{app[0]}")
  print(f"{yellow}Genres: {white}{app[1]}")
  print(f"{yellow}Android ver.: {white}{app[2]}")
  print(f"{yellow}Rating: {white}{app[3]}")
  print(red+"================================"+white)


11 apps encontradas, entre ellas:
App: Paper flowers instructions
Genres: Art & Design
Android ver.: 2.3
Rating: 4.4
App: Garden Coloring Book
Genres: Art & Design
Android ver.: 3.0
Rating: 4.4
App: 3D Color Pixel by Number - Sandbox Art Coloring
Genres: Art & Design
Android ver.: 2.3
Rating: 4.4


# Pregunta #15
[*ir al notebook dedicado*](https://colab.research.google.com/drive/16o-WrAFb9qNplC5eGw_kHop33OPwtSQ9?usp=drive_link)

> *Calcular el promedio de rating por tipo de App*

**Respuesta:** 4.17 para las gratis y 4.26 para las pagas

**Notas:**
- Como hay *una sola* línea malformada que no me sacó spark al cargar el csv, decidí dropearla a mano (por eso el `x[0] != '0'`)

In [41]:
#Nota: to_float está definida en el bloque de inicialización
apps_rdd \
  .map(lambda x: (x[0], (x[2], x[6]))) \
  .reduceByKey(lambda x,y: x) \
  .map(lambda x: (x[1][1], (to_float(x[1][0]), 1))) \
  .filter(lambda x: not math.isnan(x[1][0]) and x[0] != '0') \
  .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])) \
  .map(lambda x: (x[0], x[1][0]/x[1][1])) \
  .take(2)

[('Free', 4.166337285902508), ('Paid', 4.259602649006619)]

# Pregunta #18
[*ir al notebook dedicado*](https://colab.research.google.com/drive/1tKgaUZ9ssT79JxgLrqPjxUTKvVYhTR4O?usp=drive_link)

> *Calcule el tamaño promedio de las aplicaciones por versión de Android, sin tener en cuenta las aplicaciones que varían en tamaño según dispositivo*

**Respuesta:** *ver salida, resultados en Mb*

**Notas:**
- El `size_to_float` me convierte todos los tamaños bien: Convierte los que tienen M a megas, los que tienen K a kilos, y los que no sean un número (Varies with device y un par de filas mal formateadas) me los pone como nan
- Aún con eso sobrevivió una categoría mal formateada ('NaN), por eso el chequeo extra
- Las categorías no las toqué (ademas de la 'nan' que estaba mal), porque había preguntado al respecto por un ejercicio de pandas que también era con las categorías y me dijeron que, aparte de las líneas malas, todas las categorías las puedo tomar como "correctas"

In [42]:
def size_to_float(sz):
  try:
    char = sz[-1]
    sz = sz[:-1]
    return float(sz) if char == 'M' else float(sz)/1024
  except:
    return math.nan

ans18 = apps_rdd \
  .map(lambda x: (x[0], (x[4], x[12]))) \
  .reduceByKey(lambda x,y: x) \
  .map(lambda x: (x[1][1], (size_to_float(x[1][0]), 1))) \
  .filter(lambda x: not (math.isnan(x[1][0]) or 'NaN' in x[0])) \
  .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])) \
  .map(lambda x: (x[0], x[1][0]/x[1][1])) \
  .take(33)

# para la salida
print(f'{red}============================{white}')
for cat in ans18:
    print(f"{yellow}{cat[0]}: {white}{cat[1]:.2f} Mb")
print(f'{red}============================{white}')

4.0.3 and up: 21.22 Mb
4.2 and up: 21.15 Mb
4.4 and up: 25.27 Mb
2.3 and up: 20.26 Mb
3.0 and up: 13.73 Mb
4.1 and up: 24.28 Mb
4.0 and up: 16.49 Mb
2.2 and up: 8.22 Mb
5.0 and up: 23.31 Mb
6.0 and up: 19.46 Mb
1.6 and up: 3.11 Mb
2.1 and up: 5.58 Mb
5.1 and up: 24.06 Mb
1.5 and up: 5.07 Mb
7.0 and up: 24.31 Mb
4.3 and up: 21.92 Mb
4.0.3 - 7.1.1: 6.60 Mb
2.0 and up: 6.25 Mb
2.3.3 and up: 18.66 Mb
Varies with device: 19.91 Mb
3.2 and up: 9.86 Mb
4.4W and up: 14.13 Mb
7.1 and up: 32.00 Mb
7.0 - 7.1.1: 7.10 Mb
8.0 and up: 20.08 Mb
5.0 - 8.0: 23.80 Mb
3.1 and up: 13.41 Mb
2.0.1 and up: 22.22 Mb
4.1 - 7.1.1: 7.90 Mb
5.0 - 6.0: 11.00 Mb
1.0 and up: 3.85 Mb
2.2 - 7.1.1: 5.10 Mb
5.0 - 7.1.1: 8.40 Mb


# Pregunta #29
[*ir al notebook dedicado*](https://colab.research.google.com/drive/1Vhc3y1UGkLiGbMAW4tX9tv_2EntIPvfF?usp=drive_link)

> *Calcular la antigüedad promedio de las app de categoría “Family”*

**Respuesta:** 2212 días

**Notas:**
- Uso la misma función para castear a datetime y para asignar algún valor a todas las filas inválidas para filtrar fácil

In [43]:
def days_since(dstr):
  try:
    return (datetime.now() - datetime.strptime(dstr, "%B %d, %Y")).days
  except:
    return -1

reduced = apps_rdd \
  .filter(lambda x: "FAMILY" in x[1] and days_since(x[10]) != -1) \
  .map(lambda x: (x[0], x[10])) \
  .reduceByKey(lambda x,y: x) \
  .map(lambda x: (days_since(x[1]), 1)) \
  .reduce(lambda x, y: (x[0] + y[0], x[1] + y[1])) \

ans29 = reduced[0] / reduced[1]

# salida
print(f"Average days since last update in FAMILY category: {ans29:.0f} days")


Average days since last update in FAMILY category: 2212 days
